In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tweepy
import os




pd.set_option('display.max_columns', None)

In [2]:
def createDatasets():
    genuine_account_folders = ['data/cresci-2017/genuine_accounts.csv', 'data/cresci-2015/TFP.csv', 'data/cresci-2015/E13.csv']
    bot_account_folders = ['data/cresci-2017/social_spambots_1.csv',
                         'data/cresci-2017/social_spambots_2.csv',
                         'data/cresci-2017/social_spambots_3.csv',
                         'data/cresci-2017/traditional_spambots_1.csv']
    list_genuine_users = []
    list_genuine_tweets = []  
    for folder in genuine_account_folders:
        df_users = pd.read_csv(folder+'/users.csv',index_col='id', encoding='latin-1', low_memory=False);
        df_users['source_f'] = folder; 
        list_genuine_users.append(df_users);
        df_tweets = pd.read_csv(folder+'/tweets.csv',index_col='id', encoding='latin-1', low_memory=False);
        df_tweets['source_f'] = folder; 
        list_genuine_tweets.append(df_tweets);
        genuine_tweets = pd.concat(list_genuine_tweets)
        genuine_users = pd.concat(list_genuine_users)
    
    list_bot_users = []
    list_bot_tweets = []
    for folder in bot_account_folders:
        df_users = pd.read_csv(folder+'/users.csv',index_col='id', encoding='latin-1', low_memory=False);
        df_users['source_f'] = folder;
        list_bot_users.append(df_users);    
        df_tweets = pd.read_csv(folder+'/tweets.csv',index_col='id', encoding='latin-1', low_memory=False);
        df_tweets['source_f'] = folder;
        list_bot_tweets.append(df_tweets);
        bot_tweets = pd.concat(list_bot_tweets)
        bot_users = pd.concat(list_bot_users)

    users = pd.concat([bot_users,genuine_users])
    tweets = pd.concat([bot_tweets,genuine_tweets])

    user_class = pd.concat([pd.DataFrame(np.ones(shape=(len(bot_users),1))),pd.DataFrame(np.zeros(shape=(len(genuine_users),1)))])
    user_class.rename(columns={0:'is_bot'},inplace=True);

  # Remove users that have no associated tweets
    user_class = user_class[users.index.isin(tweets.set_index('user_id').index)]
    users = users[users.index.isin(tweets.set_index('user_id').index)]

  ## Delete empty values
    del users['contributors_enabled'];
    del users['follow_request_sent'];
    del users['following'];
    del users['notifications'];
    del users['test_set_1'];
    del users['test_set_2'];
  
    del tweets['geo'];
    del tweets['contributors'];
    del tweets['favorited'];
    del tweets['retweeted'];

    ## Update timestamp values to be datetime
    tweets['timestamp_dt'] = pd.to_datetime(tweets['timestamp'],format='%Y-%m-%d %H:%M:%S')
    del tweets['timestamp']

    return users, user_class, tweets                                   

In [3]:
users, user_class, tweets = createDatasets()

c:\python\python385\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
tweets

,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,place,retweet_count,reply_count,favorite_count,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,crawled_at,updated,source_f,timestamp_dt
id,,,,,,,,,,,,,,,,,,,,,
532627591686275072,I Pooh - In silenzio 1968 http://t.co/ahvQxUqTws,"<a href=""http://www.facebook.com/twitter"" rel=...",2.485829e+07,NaN,0.000000e+00,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,1.0,0.0,Wed Nov 12 20:14:48 +0000 2014,2014-11-12 21:44:09,2014-11-12 21:44:09,data/cresci-2017/social_spambots_1.csv,2014-11-12 21:14:48
532624255058706432,http://t.co/HyI5EQKz6Q,"<a href=""http://www.facebook.com/twitter"" rel=...",2.485829e+07,NaN,0.000000e+00,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,1.0,0.0,Wed Nov 12 20:01:32 +0000 2014,2014-11-12 21:44:09,2014-11-12 21:44:09,data/cresci-2017/social_spambots_1.csv,2014-11-12 21:01:32
532513524460052480,"Tutti a tavola, con il filetto di baccalÃ . ht...","<a href=""http://www.facebook.com/twitter"" rel=...",2.485829e+07,NaN,0.000000e+00,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,1.0,0.0,Wed Nov 12 12:41:32 +0000 2014,2014-11-12 21:44:09,2014-11-12 21:44:09,data/cresci-2017/social_spambots_1.csv,2014-11-12 13:41:32
532297646669852672,http://t.co/NAHQ4l2pUy,"<a href=""http://www.facebook.com/twitter"" rel=...",2.485829e+07,NaN,0.000000e+00,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,1.0,0.0,Tue Nov 11 22:23:43 +0000 2014,2014-11-12 21:44:09,2014-11-12 21:44:09,data/cresci-2017/social_spambots_1.csv,2014-11-11 23:23:43
532295960807100416,Gold - Spandau Ballet http://t.co/o8ZJHt7Neu,"<a href=""http://www.facebook.com/twitter"" rel=...",2.485829e+07,NaN,0.000000e+00,0.0,NaN,0.0,NaN,0.0,0.0,0.0,NaN,0.0,1.0,0.0,Tue Nov 11 22:17:01 +0000 2014,2014-11-12 21:44:09,2014-11-12 21:44:09,data/cresci-2017/social_spambots_1.csv,2014-11-11 23:17:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306157087241678848,RT @Radio_Italia: #solopervoi uno 'sguardo' sp...,web,1.219743e+09,NaN,0.000000e+00,0.0,NaN,NaN,NaN,13.0,NaN,0.0,NaN,1.0,1.0,2.0,Mon Feb 25 21:41:52 +0000 2013,NaN,NaN,data/cresci-2015/E13.csv,2013-02-25 22:41:52
306157022741676032,@xkissmeharold Coraggio sorÃ¨!,web,1.219743e+09,NaN,3.061525e+17,298545638.0,xkissmeharold,NaN,NaN,1.0,NaN,1.0,NaN,0.0,0.0,1.0,Mon Feb 25 21:41:37 +0000 2013,NaN,NaN,data/cresci-2015/E13.csv,2013-02-25 22:41:37
306147626896736256,RT @RollingStoneIta: #GeorgeHarrison avrebbe c...,web,1.219743e+09,NaN,0.000000e+00,0.0,NaN,NaN,NaN,13.0,NaN,0.0,NaN,1.0,1.0,1.0,Mon Feb 25 21:04:16 +0000 2013,NaN,NaN,data/cresci-2015/E13.csv,2013-02-25 22:04:16


In [19]:
def normalizedInput(users, tweets, trainFlag=0):
    ## Add friend-follower ratio as a feature
    users.loc[:,'friend_follower_ratio'] = users['friends_count']/(users['followers_count'] + 1) ## To avoid divide by zero errors
    
    ## Add average-per-tweet information
    averages = tweets[['num_hashtags','retweet_count','favorite_count','num_mentions','num_urls','user_id']].groupby('user_id').mean()
    ## Reply count is sometimes blank -- replace with 0's
    averages = averages.fillna(0)
    users = users.join(averages.add_suffix('_per_tweet'))
    
    ## Add number of unique places where the user has tweeted.  
    # This will be 0 if no places have ever been tagged
    users.loc[:,'unique_tweet_places'] = tweets.groupby('user_id').place.nunique()

    ## Add: Variance in the user's number of tweets per hour.
    tweets.loc[:,'date'] = tweets.timestamp_dt.dt.date
    tweets.loc[:,'hour'] = tweets.timestamp_dt.dt.hour
    variance_in_tweet_rate = tweets.groupby(['user_id','date','hour']).size().groupby('user_id').var()
    variance_in_tweet_rate.rename('variance_in_tweet_rate',inplace=True)
    users = users.join(variance_in_tweet_rate)

    ## Feature selection code should go here (Think About this!)
    X = users[['favourites_count','followers_count','friends_count','verified','friend_follower_ratio','num_hashtags_per_tweet','retweet_count_per_tweet','favorite_count_per_tweet','num_mentions_per_tweet','num_urls_per_tweet','unique_tweet_places','variance_in_tweet_rate']]
    
    values = {'verified': 0, 'variance_in_tweet_rate': 0}
    X = X.fillna(value=values)
    
    if trainFlag == 1:
        scaler = StandardScaler()
        scaler = scaler.fit(X)
        pickle.dump(scaler, open("Pickle/scaler.p","wb"))
        X.loc[:,:] = scaler.transform(X)
    else:
        scaler = pickle.load(open("Pickle/scaler.p","rb"))
        X.loc[:,:] = scaler.transform(X)
    
    return X

In [21]:
X = normalizedInput(users, tweets, 1)

In [28]:
X_train, X_train_dev, y_train, y_train_dev = train_test_split(X, user_class, test_size=0.2)
    
pickle.dump(X_train, open( "Pickle/X_train.p", "wb" ))
pickle.dump(X_train_dev, open( "Pickle/X_train_dev.p", "wb" ))
pickle.dump(y_train, open( "Pickle/y_train.p", "wb" ))
pickle.dump(y_train_dev, open( "Pickle/y_train_dev.p", "wb" ))

In [31]:
def connect_to_api():
    auth = tweepy.OAuthHandler(settings.consumer_key, settings.consumer_secret)
    auth.set_access_token(settings.access_token, settings.access_token_secret)
    api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    return api

In [32]:
def loadTestData():
    
    filename = 'data/varol-2017.dat'
    df = pd.read_csv(filename, sep='\t',header = None)
    
    df.rename(columns={0: 'id', 1: 'is_bot'},inplace=True)
    
    df.set_index('id',inplace=True)
    
    return df